In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-05-17 01:08:42--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   131MB/s    in 0.6s    

2021-05-17 01:08:43 (131 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from rdkit import Chem

In [ ]:
df = pd.read_csv('delaney_edited_F1.csv')
df

,Compound ID,measured log(solubility:mol/L),ESOL predicted log(solubility:mol/L),Human Intestinal Absorption,Blood Brain Barrier,Caco-2 permeable,P-glycoprotein substrate,Ames test,Carcinogenicity,Biodegradation,Rat acute toxicity (mol/kg),SMILES
0,"1,1,1,2-Tetrachloroethane",-2.180,-2.794,1,1,1,0,0,0,1,1.8596,ClCC(Cl)(Cl)Cl
1,"1,1,1-Trichloroethane",-2.000,-2.232,1,1,1,1,0,0,1,2.6730,CC(Cl)(Cl)Cl
2,"1,1,2,2-Tetrachloroethane",-1.740,-2.549,1,1,1,1,0,0,1,2.1234,ClC(Cl)C(Cl)Cl
3,"1,1,2-Trichloroethane",-1.480,-1.961,1,1,1,1,0,0,1,1.2334,ClCC(Cl)Cl
4,"1,1,2-Trichlorotrifluoroethane",-3.040,-3.077,1,1,1,1,0,0,1,3.2234,FC(F)(Cl)C(F)(Cl)Cl
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1-Methylnaphthalene,-3.700,-3.802,1,1,1,1,0,0,1,2.2224,Cc1cccc2ccccc12
96,1-Methylphenanthrene,-5.850,-4.870,1,1,1,0,0,0,1,1.9974,Cc1cccc2c1ccc3ccccc32
97,1-methyluracil,-0.807,-0.375,1,1,1,1,0,0,1,2.4353,Cn1ccc(=O)[nH]c1=O
98,1-Napthol,-2.220,-3.080,1,1,1,0,0,0,1,1.0093,Oc1cccc2ccccc12


In [ ]:
df.SMILES

0            ClCC(Cl)(Cl)Cl
1              CC(Cl)(Cl)Cl
2            ClC(Cl)C(Cl)Cl
3                ClCC(Cl)Cl
4       FC(F)(Cl)C(F)(Cl)Cl
              ...          
95          Cc1cccc2ccccc12
96    Cc1cccc2c1ccc3ccccc32
97       Cn1ccc(=O)[nH]c1=O
98          Oc1cccc2ccccc12
99          Oc1cccc2ccccc12
Name: SMILES, Length: 100, dtype: object

In [ ]:
from rdkit.Chem import Descriptors

In [ ]:
mol_list= []
for element in df.SMILES:
  mol = Chem.MolFromSmiles(element)
  mol_list.append(mol)

In [ ]:
def generate(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_MolWt = Descriptors.MolWt(mol)
        desc_NumRotatableBonds = Descriptors.NumRotatableBonds(mol)
           
        row = np.array([desc_MolLogP,
                        desc_MolWt,
                        desc_NumRotatableBonds])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MolLogP","MolWt","NumRotatableBonds"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [ ]:
df1 = generate(df.SMILES)

In [ ]:
def AromaticAtoms(m):
  aromatic_atoms = [m.GetAtomWithIdx(i).GetIsAromatic() for i in range(m.GetNumAtoms())]
  aa_count = []
  for i in aromatic_atoms:
    if i==True:
      aa_count.append(1)
  sum_aa_count = sum(aa_count)
  return sum_aa_count

In [ ]:
desc_AromaticProportion = [AromaticAtoms(element)/Descriptors.HeavyAtomCount(element) for element in mol_list]
desc_AromaticProportion


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6,
 0.6,
 0.6,
 0.6666666666666666,
 0.6666666666666666,
 0.6,
 0.6,
 0.6,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.75,
 0.75,
 0.0,
 0.75,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6,
 0.5,
 0.0,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.4,
 0.75,
 0.0,
 0.75,
 0.75,
 0.0,
 0.0,
 0.75,
 0.8333333333333334,
 0.5,
 0.75,
 0.0,
 0.75,
 0.75,
 0.6,
 0.75,
 0.8333333333333334,
 0.5,
 0.0,
 0.8333333333333334,
 0.0,
 1.0,
 0.0,
 0.0,
 0.9333333333333333,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9090909090909091,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9090909090909091,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8333333333333334,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9090909090909091,
 0.0,
 0.0,
 0.8571428571428571,
 0.9090909090909091,
 0.9333333333333333,
 0.6666666666666666,
 0.9090909090909091,
 0.9090909090909091]

In [ ]:
df_desc_AromaticProportion = pd.DataFrame(desc_AromaticProportion, columns=['AromaticProportion'])
df_desc_AromaticProportion

,AromaticProportion
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
95,0.909091
96,0.933333
97,0.666667
98,0.909091


In [ ]:
df1

,MolLogP,MolWt,NumRotatableBonds
0,2.59540,167.850,0.0
1,2.37650,133.405,0.0
2,2.59380,167.850,1.0
3,2.02890,133.405,1.0
4,2.91890,187.375,1.0
...,...,...,...
95,3.14822,142.201,0.0
96,4.30142,192.261,0.0
97,-0.92640,126.115,0.0
98,2.54540,144.173,0.0


In [ ]:
X = pd.concat([df1,df_desc_AromaticProportion], axis=1)
X

,MolLogP,MolWt,NumRotatableBonds,AromaticProportion
0,2.59540,167.850,0.0,0.000000
1,2.37650,133.405,0.0,0.000000
2,2.59380,167.850,1.0,0.000000
3,2.02890,133.405,1.0,0.000000
4,2.91890,187.375,1.0,0.000000
...,...,...,...,...
95,3.14822,142.201,0.0,0.909091
96,4.30142,192.261,0.0,0.933333
97,-0.92640,126.115,0.0,0.666667
98,2.54540,144.173,0.0,0.909091


In [ ]:
df.head()

,Compound ID,measured log(solubility:mol/L),ESOL predicted log(solubility:mol/L),Human Intestinal Absorption,Blood Brain Barrier,Caco-2 permeable,P-glycoprotein substrate,Ames test,Carcinogenicity,Biodegradation,Rat acute toxicity (mol/kg),SMILES
0,"1,1,1,2-Tetrachloroethane",-2.18,-2.794,1,1,1,0,0,0,1,1.8596,ClCC(Cl)(Cl)Cl
1,"1,1,1-Trichloroethane",-2.00,-2.232,1,1,1,1,0,0,1,2.6730,CC(Cl)(Cl)Cl
2,"1,1,2,2-Tetrachloroethane",-1.74,-2.549,1,1,1,1,0,0,1,2.1234,ClC(Cl)C(Cl)Cl
3,"1,1,2-Trichloroethane",-1.48,-1.961,1,1,1,1,0,0,1,1.2334,ClCC(Cl)Cl
4,"1,1,2-Trichlorotrifluoroethane",-3.04,-3.077,1,1,1,1,0,0,1,3.2234,FC(F)(Cl)C(F)(Cl)Cl


In [ ]:
y = df.iloc[:,1:10]
y

,measured log(solubility:mol/L),ESOL predicted log(solubility:mol/L),Human Intestinal Absorption,Blood Brain Barrier,Caco-2 permeable,P-glycoprotein substrate,Ames test,Carcinogenicity,Biodegradation
0,-2.180,-2.794,1,1,1,0,0,0,1
1,-2.000,-2.232,1,1,1,1,0,0,1
2,-1.740,-2.549,1,1,1,1,0,0,1
3,-1.480,-1.961,1,1,1,1,0,0,1
4,-3.040,-3.077,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...
95,-3.700,-3.802,1,1,1,1,0,0,1
96,-5.850,-4.870,1,1,1,0,0,0,1
97,-0.807,-0.375,1,1,1,1,0,0,1
98,-2.220,-3.080,1,1,1,0,0,0,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
y_pred_train = model.predict(X_train)

In [ ]:
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_train, y_pred_train))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_train, y_pred_train))


Coefficients: [[-1.05939371e+00 -3.32832221e-03 -1.17566043e-01 -4.93133939e-01]
 [-8.37372427e-01 -3.96962189e-03  2.13205329e-02 -8.11194746e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-9.05171006e-02  7.78030730e-04  9.02188591e-03 -5.24129735e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
Intercept: [0.64529294 0.09903342 1.         1.         1.         0.61239854
 0.         0.         1.        ]
Mean squared error (MSE): 0.09
Coefficient of determination (R^2): 0.87


In [ ]:
y_pred_test = model.predict(X_test)

In [ ]:
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(y_test, y_pred_test))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(y_test, y_pred_test))


Coefficients: [[-1.05939371e+00 -3.32832221e-03 -1.17566043e-01 -4.93133939e-01]
 [-8.37372427e-01 -3.96962189e-03  2.13205329e-02 -8.11194746e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-9.05171006e-02  7.78030730e-04  9.02188591e-03 -5.24129735e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
Intercept: [0.64529294 0.09903342 1.         1.         1.         0.61239854
 0.         0.         1.        ]
Mean squared error (MSE): 0.06
Coefficient of determination (R^2): 0.86
